In [0]:
## IMPORTANDO BIBLIOTECAS ###########################################################################################################

from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as F
import re
from datetime import datetime ,timedelta

In [0]:
## CONFIGURAÇÕES AMBIENTE ############################################################################################################

spark.conf.set("spark.sql.legacy.timeParserPolicy", "Legacy") # para schemas de parquet diferente

In [0]:
## VARIÁVEIS DE EXECUÇÃO DO PROCESSO ################################################################################################

dict_PARAMETRO = {
      'Caminho_parquet'  : '/FileStore/tables/dados/OriginaisNetflix.parquet'
     ,'Caminho' : '/FileStore/tables/dados/CSV/'
                 }

vCAMINHO = dict_PARAMETRO['Caminho']
vCAMINHOARQUIVO = dict_PARAMETRO['Caminho_parquet']


## VARIAVEIS DE REGRA ##############################################################################################################

dict_MES =  {"jan" : "01"
            ,"feb" : "02"
            ,"mar" : "03"
            ,"apr" : "04"
            ,"may" : "05"
            ,"jun" : "06"
            ,"jul" : "07"
            ,"aug" : "08"
            ,"sep" : "09"
            ,"oct" : "10"
            ,"nov" : "11"
            ,"dec" : "12"}

In [0]:
## INICIO TRATATIVA DE DADOS ######################################################################################################

df_TEMP = spark.read.parquet(vCAMINHOARQUIVO,inferSchema=True)

## ALTERANDO CAMPOS "Premiere" e "dt_inclusao" DE STRING PARA DATETIME ############################################################

for vMES in list(dict_MES):
    df_TEMP = df_TEMP.withColumn('Premiere',F.regexp_replace(F.lower(df_TEMP['Premiere']), vMES, dict_MES[vMES]))
    
df_TEMP = (df_TEMP.withColumn('Premiere',F.to_timestamp(F.to_date(df_TEMP['Premiere'],'dd-MM-yy'),'yyyy-MM-dd hh:mm:ss'))
                  .withColumn('dt_inclusao',F.to_utc_timestamp(df_TEMP['dt_inclusao'],'UTC+03:00')))

## CLASSIFICANDO PELA COLUNA "ACTIVE" E PELA COLUNA "GENRE" ########################################################################

df_TEMP = df_TEMP.sort(df_TEMP.Active.desc(),df_TEMP.Genre.desc())

## REMOVENDO DUPLICATAS ############################################################################################################

df_TEMP = df_TEMP.dropDuplicates()

## TROCANDO RESULTADOS DA COLUNA "SEASONS" DE "TBA" PARA "A SER ANUNCIADO" ########################################################

df_TEMP = df_TEMP.withColumn('Seasons',F.regexp_replace(df_TEMP['Seasons'], 'TBA', 'a ser anunciado'))

## CRIANDO COLUNA DATA DE ALTERAÇÃO ###############################################################################################

df_TEMP = df_TEMP.withColumn('Data de Alteração',F.lit(datetime.now()))

## RENOMEANDO COLUNAS PARA PORTUGUES ##############################################################################################

df_FINAL = (df_TEMP.withColumnRenamed("Title","Título")
                 .withColumnRenamed("Genre","Gênero")
                 .withColumnRenamed("GenreLabels","Etiqueta de Gênero")
                 .withColumnRenamed("Premiere","Estréia")
                 .withColumnRenamed("Seasons","Temporadas")
                 .withColumnRenamed("SeasonsParsed","Temporadas Analisadas")
                 .withColumnRenamed("EpisodesParsed","Episódios Analisadas")
                 .withColumnRenamed("Length","Duração")
                 .withColumnRenamed("MinLength","Duração Mínima")
                 .withColumnRenamed("MaxLength","Duração Máxima")
                 .withColumnRenamed("Status","Situação")
                 .withColumnRenamed("Active","Ativo")
                 .withColumnRenamed("Table","Categoria")
                 .withColumnRenamed("Language","Idioma")
                 .withColumnRenamed("dt_inclusao","Data Inclusão")
          )

In [0]:
## CRIANDO ARQUIVO CSV E SALVANDO DENTRO DO BUCKET ##############################################################################

df_FINAL.write.format("com.databricks.spark.csv").mode("overwrite").option("header", "true").save(vCAMINHO+'OriginaisNetflix.csv')